# Decision Trees

## Training and Visualizing a Decision Tree

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:] # petal length and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

DecisionTreeClassifier(max_depth=2)

In [2]:
from sklearn.tree import export_graphviz

export_graphviz(
    tree_clf,
    out_file=image_path("iris_tree.dot"),
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

NameError: name 'image_path' is not defined

`$ dot -Tpng iris_tree.dot -o iris_tree.png`

## Making Predictions

`samples` attribute counts how many training instances it applies to <br>
`value` attribute tells you how many training instances of each class this node applies to <br>
`gini` attribute measures its *impurity* ("pure" if gini=0)

Gini impurity

$G_{i} = 1 - \sum_{k=1}^{n}p_{i,k}^{2}$

$p_{i,k}$ is the ratio of class k instances among the training instances in the ith node

### Model Interpretation: White Box vs Black Box

Decision Trees are intuitive and their decisions are easy to interpret. Such models are often called *white box models*. In contrast, Random Forests or neural networks are considered *black box models*. They make great predictions and the calculations they perform are easy to check, but it is hard to explain why the predictioins were made. Conversely, Decision Trees provide nice, simple classification rules.

## Estimating Class Probabilities

Decision trees also estimate the probability that an instance belongs to a particualr class k by returning the ratio of training instances of class k in a particular node. 

In [ ]:
>>> tree_clf.predict_proba([[5, 1.5]])

In [ ]:
>>> tree_clf.predict([[5, 1.5]])

## The CART Training Algorithm

Scikit-Learn uses the *Classification and Regression Tree* (CART) algorithm to train Decision Trees (also called "growing" trees). Algorithm works by first splitting the training set into two subsets using a single feature $k$ and a threshold $t_{k}$. It finds $t$ and $t_{k}$ by searching for the pair that produces the purest subsets (weighted by their size). 

CART cost function for classification

$J(t,t_{k}) = \frac{m_{left}}{m}G_{left} + \frac{m_{right}}{m}G_{right}$

Once the CART algorithm has successfully split the training set in two, it splits the subsets using the same logic, then the sub-subsets, and so on, recursively. It stops recursing once it reaches the maximum depth (defined by `max_depth`) or if it cannot find a split that will reduce impurity. 

CART is a greedy algorithm. Most of the time, it will not produce a solution with the lowest possible impurity (not guranteed to be optimal). This is because finding the optimal tree is considered an NP-Complete problem. Therefore, we must settle for a reasonably good solution. 

## Computational Complexity

Making predictions required traversing the Decision Tree from the root to a leaf. Decision Trees are roughly balanced, so runtime is roughly $O(log_{2}(m))$. Since each node only requires checking the value of one feature, the overall prediction complexity is $O(log_{2}(m))$

Training compares all features which is $O(n*mlog_{2}(m))$

## Gini Impurity or Entropy?

In ML, *entropy* is often used as an impurity measure after Shannon's *information theory*, where it measures the average information content of a message: entropy is zero when al messages are identical. By default Gini impurity measure is used. To switch to entropy, set `criterion` hyperparameter to `"entropy"`

Entropy

$H_{i} = - \sum_{k=1}^n p_{i,k} log_{2}(p_{i,k})$
<br>
such that $p_{i,k}$ != 0

Gini is slightly faster to. When they differ, Gini tends to isolate the most frequent class in its own branch of the tree, while entropy tends to produce slightly more balanced trees. 

## Regularization Hyperparameters

Decision Trees make very few assumptions about training data, so they are likely to fit it every well. This can lead to overfitting. This type of model is called a *nonparametric model* because the number of parameters is not determined prior to training, so the model structure is free to stick closely to the data. In contrast, a *parametric model*, such as a linear model, has predetermined number of parameters, so its degree of freedom is limited.

Restrict Decision Tree's freedom during druing to avoid overfitting (regularization). In Scikit-Learn, reduce `max_depth` hyperparamter to regularize the model and reduce risk of overfitting. 

`DecisionTreeClassifier` class has other parameters that restrict the shape of the Decision Tree: <br>
`min_samples_leaf` - the minimum number of samples a leaf node must have) <br>
`min_weight_fraction_leaf` - the same as `min_samples_leaf` but expressed as a fraction of the total number of weighted instances <br>
`max_leaf_nodes` - the maximum number of leaf nodes <br>
`max_features` - the maximum number of features that are evalutated for splitting at each node <br>
Increasing `min_*` hyperparameters or reducing `max_*` hyperparameters will regularize

Other algorithms work by first training Decision Tree without restrictions, then *pruning* (deleting) unnecessary nodes. A node whose children are all leaf nodes is considered unnecessary if the purity improvement it provides is not statistically significant. Use $\chi^{2}$ test (chi-squared test) to estimate the probability that the improvement is purely the result of chance (null hypothesis). If the probability is higher than the set *p-value*, a given threshold, then the node is considered unnecessary and its children are deleted

## Regression

`DecisionTreeRegressor`

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)

Instead of predicting a class in each node, it predicts a value - the average target value of the m training instances associated with this leaf node, and it results in a mean squared error over the instances

CART works the same way, but instead of trying to split the training set in a way that minimizes impurity, it tries to split the training set in a way that minimizes the MSE.

CART cost function for regression

$J(k,t_{k}) = \frac{m_{left}}{m}MSE_{left} + \frac{m_{right}}{m}MSE_{right}$

where

$MSE_{node} = \sum_{i ∈ node} (\hat{y}_{node} - y^{(i)})^{2}$

$\hat{y} = \frac{1}{m_{node}} \sum_{i ∈ node} y^{(i)}$

Just like classification, decision trees for regression are prone to overfitting without regularization

## Instability

Decision Trees are powerful, but have some limitations:

Decision Trees love orthogonal decision boundaries (all splits are perpendicular to an axis); this makes them sensitive to training set rotation

Biggest weakness is that they're very sensitive to small variations in the training data. For example, removing the widest *Iris versicolor* from the iris training set and training a new Decision Tree, gives you a very different model. Random Forests limit this instability by averaging predictions over many trees

# Exercises

### 1. What is the approximate depth of a Decision Tree training (without retstrictions) on a training set with one million instances?
<br>
$log_{2}$(1 million) = 20

### 2. Is a node's Gini impurity generally lower or greater than its parent's? Is it *generally* lower/greater, or *always* lower/greater?
<br>
It is generally lower. Because growing a tree is an NP complete problem, we settle for using a Greedy algorithm that attempts to minimize the gini at each possible depth. This doesn't always happen (because of the greedy approach is not optimal) so it may get stuck at a local minimum before breaking out and finding the true minimum.
<br><br>
*It is possible for a node to have a higher Gini impurity than its parent, as long as this increase is more than compensated for by a decrease in the other child's impurity*

### 3. If a Decision Tree is overfitting the training set, is it a good idea to try decreasing `max_depth`?
<br>
Yes

### 4. If a Decision Tree is underfitting the training set, is it a good idea to try scaling the input features?
<br>
Scaling is always a good idea regardless of under or overfitting data
<br><br>
*Decision trees don't care about scaling or centering the data*

### 5. If it takes one hour to train a Decision Tree on a training set containing 1 million instances, roughly how much time will it take to train another Decision Tree on a training set containing 10 million instances?
<br>
$O(n*m*log(m))$
<br><br>
K = $\frac{n*10m*log(10*m)}{n*m*log(m)} = \frac{10*log(10m)}{log(m)}$
<br><br>
K = $\frac{10*log(10*10^{6})}{log(10^{6})}$ = 11.7 hours

### 6. If your training set contains 100,000 instances, will setting `presort=True` speed up training?
<br>
That may be small enough to speed up training. If the training set was larger, it would probably slow it down.
<br><br>
*presorting is only useful if the training set is smaller than a few thousand. That amount will considerably slow down training*

### 7. Train and fine-tune a Decision Tree for the moons dataset by following the following steps:
a. Use `make_moons(n_samples=10000, noise=0.4)` to generate a moons dataset <br>
b. Use `train_test_split()` to split the dataset into a training set and a test set <br>
c. Use grid search with cross-validation (with the help of the `GridSearchCV` class) to find good hyperparameter values for a `DecisionTreeClassifier`. Hint: try various values for `max_leaf_nodes` <br>
d. Train it on the full training set using these hyperparameters, and measure your model's performance on the test set. You should roughly get 85% to 87% accuracy

In [4]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=10000, noise=0.4)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

tree_clf = DecisionTreeClassifier(random_state=42)

param_grid = {
    'max_depth': [x for x in range(2, 10, 2)],
    'max_leaf_nodes': [x for x in range(2, 50, 2)]
}

tree_grid = GridSearchCV(tree_clf, param_grid, cv=3, verbose=2)
tree_grid.fit(X_train, y_train)

Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] END ......................max_depth=2, max_leaf_nodes=2; total time=   0.0s
[CV] END ......................max_depth=2, max_leaf_nodes=2; total time=   0.0s
[CV] END ......................max_depth=2, max_leaf_nodes=2; total time=   0.0s
[CV] END ......................max_depth=2, max_leaf_nodes=4; total time=   0.0s
[CV] END ......................max_depth=2, max_leaf_nodes=4; total time=   0.0s
[CV] END ......................max_depth=2, max_leaf_nodes=4; total time=   0.0s
[CV] END ......................max_depth=2, max_leaf_nodes=6; total time=   0.0s
[CV] END ......................max_depth=2, max_leaf_nodes=6; total time=   0.0s
[CV] END ......................max_depth=2, max_leaf_nodes=6; total time=   0.0s
[CV] END ......................max_depth=2, max_leaf_nodes=8; total time=   0.0s
[CV] END ......................max_depth=2, max_leaf_nodes=8; total time=   0.0s
[CV] END ......................max_depth=2, max

[CV] END .....................max_depth=4, max_leaf_nodes=20; total time=   0.0s
[CV] END .....................max_depth=4, max_leaf_nodes=22; total time=   0.0s
[CV] END .....................max_depth=4, max_leaf_nodes=22; total time=   0.0s
[CV] END .....................max_depth=4, max_leaf_nodes=22; total time=   0.0s
[CV] END .....................max_depth=4, max_leaf_nodes=24; total time=   0.0s
[CV] END .....................max_depth=4, max_leaf_nodes=24; total time=   0.0s
[CV] END .....................max_depth=4, max_leaf_nodes=24; total time=   0.0s
[CV] END .....................max_depth=4, max_leaf_nodes=26; total time=   0.0s
[CV] END .....................max_depth=4, max_leaf_nodes=26; total time=   0.0s
[CV] END .....................max_depth=4, max_leaf_nodes=26; total time=   0.0s
[CV] END .....................max_depth=4, max_leaf_nodes=28; total time=   0.0s
[CV] END .....................max_depth=4, max_leaf_nodes=28; total time=   0.0s
[CV] END ...................

[CV] END .....................max_depth=6, max_leaf_nodes=48; total time=   0.0s
[CV] END .....................max_depth=6, max_leaf_nodes=48; total time=   0.0s
[CV] END ......................max_depth=8, max_leaf_nodes=2; total time=   0.0s
[CV] END ......................max_depth=8, max_leaf_nodes=2; total time=   0.0s
[CV] END ......................max_depth=8, max_leaf_nodes=2; total time=   0.0s
[CV] END ......................max_depth=8, max_leaf_nodes=4; total time=   0.0s
[CV] END ......................max_depth=8, max_leaf_nodes=4; total time=   0.0s
[CV] END ......................max_depth=8, max_leaf_nodes=4; total time=   0.0s
[CV] END ......................max_depth=8, max_leaf_nodes=6; total time=   0.0s
[CV] END ......................max_depth=8, max_leaf_nodes=6; total time=   0.0s
[CV] END ......................max_depth=8, max_leaf_nodes=6; total time=   0.0s
[CV] END ......................max_depth=8, max_leaf_nodes=8; total time=   0.0s
[CV] END ...................

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [2, 4, 6, 8],
                         'max_leaf_nodes': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20,
                                            22, 24, 26, 28, 30, 32, 34, 36, 38,
                                            40, 42, 44, 46, 48]},
             verbose=2)

In [20]:
tree_grid.best_params_

{'max_depth': 8, 'max_leaf_nodes': 24}

In [21]:
best_grid = tree_grid.best_estimator_
best_grid.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=8, max_leaf_nodes=24, random_state=42)

In [23]:
from sklearn.metrics import accuracy_score

y_pred = best_grid.predict(X_train)
accuracy_score(y_pred, y_train)

0.869375

In [25]:
y_pred = best_grid.predict(X_test)
accuracy_score(y_pred, y_test)

0.8535

### 8. Grow a forest by following these steps:
a. Continue the previous exercise, generate 1000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit-Learn's `ShuffleSplit` class for this. <br>
b. Train one Decision Tree on each subset, using the best hyperparameter values found in the previous exercise. Evaluate these 1000 Decision Trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely perform worse than the first Decision Tree, achieving only about 80% accuracy.<br>
c. Now comes the magic. For each test set instance, generate the predictions of the 1000 Decision Trees, and keep only the most frequent predicition (you can use SciPy's `mode()` function for this). This approach gives you *majority-vote predictions* over the test set.<br>
d. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a Random Forest classifier!

In [43]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
for train_index, test_index in rs.split(X_train):
    X_mini = X_train[train_index]
    y_mini = y_train[train_index]
    mini_sets.append((X_mini, y_mini))

In [44]:
from sklearn.base import clone
import numpy as np

forest = [clone(tree_grid.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini, y_mini) in zip(forest, mini_sets):
    tree.fit(X_mini, y_mini)
    
    y_predict = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_pred, y_test))

np.mean(accuracy_scores)

0.8535000000000003

In [45]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

In [46]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [47]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.8545